In [1]:
import pandas as pd

In [2]:
compra_saldo = pd.read_csv("../db/compra_saldo_sep.csv")
liquidez = pd.read_csv("../db/liquidez_sep.csv")
compra_saldo.head(3)

,KnNoPrecalificacion,DxIniciativa,DxDpi,DxTipoCliente,DxNombreCompletoCliente,DdFechaVencimiento,KdFechaCarga,DxClienteBI,DxConsolidaBI,DdFechaIngreso,...,DxGenero,DxPais,DxDepartamento,DxMunicipio,DfRciGlobalInicialDeudor,DfRciInternoInicialDeudor,DfIngresosValidados,DfMontoMaximoOfertado,DnCantidadCreditosExternosConsolidados,DfSaldoTotalCreditosExternosConsolidados
0,1572862740805,CARTERA_ACTIVA_SEPTIEMBRE_2024,1572862740805,CLIENTE EXISTENTE,ELVIA MAGDALENA PEREZ AJTUN,2024-12-01T00:00:00.0000000,2024-10-17,NO,NO,1995-01-02,...,F,GUATEMALA,TOTONICAPAN,MOMOSTENANGO,59.28,35.92,6891.500,22500,1,18528.310547
1,1572880131326,CARTERA_ACTIVA_SEPTIEMBRE_2024,1572880131326,CLIENTE EXISTENTE,DELIA ESTER MARTINEZ GONZALEZ,2024-12-01T00:00:00.0000000,2024-10-17,NO,NO,2018-09-20,...,F,GUATEMALA,HUEHUETENANGO,HUEHUETENANGO,46.83,5.69,6891.500,150000,1,79950.851562
2,1572882260101,CARTERA_ACTIVA_SEPTIEMBRE_2024,1572882260101,CLIENTE EXISTENTE,WENDI MARISOL PULUC MUNIZ,2024-12-01T00:00:00.0000000,2024-10-17,NO,NO,2007-10-17,...,F,GUATEMALA,GUATEMALA,GUATEMALA,29.03,3.07,6945.306,170000,1,16165.650391


In [3]:
cartera = pd.read_csv("../db/cartera.csv", sep=";")
cartera = cartera[cartera["RemesaActual"] == 1]
cartera_temp = cartera.groupby(by="DPI")["SaldoCapital"].sum().reset_index()
cartera_temp.columns = ["DPI", "SaldoInterno"]
cartera = pd.merge(cartera, cartera_temp, how="inner", on="DPI")
cartera.to_csv("../db/SaldosInternos.csv")
deuda_interna = pd.read_csv("../db/SaldosInternos.csv")

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_131076\1983517603.py:1: DtypeWarning: Columns (43,45,46,61,63,68) have mixed types. Specify dtype option on import or set low_memory=False.
  cartera = pd.read_csv("../db/cartera.csv", sep=";")


In [4]:
compra_saldo = pd.merge(compra_saldo, deuda_interna, how="inner", left_on="DxDpi", right_on="DPI")
liquidez = pd.merge(liquidez, deuda_interna, how="inner", left_on="DxDpi", right_on="DPI")

In [5]:
compra_saldo = compra_saldo[["DPI", "SaldoExterno", "SaldoInterno",
                             "RciGlobalInicial", "RciInternoInicial", 
                             "ScoreInstallmentCat", "ScoreInstallment"]]

liquidez = liquidez[["DPI", "SaldoExterno", "SaldoInterno",
                             "RciGlobalInicial", "RciInternoInicial", 
                             "ScoreInstallmentCat", "ScoreInstallment"]]

## Calcular lo solicitado

In [6]:
todos = cartera[["DPI", "SaldoExterno", "SaldoInterno",
                             "RciGlobalInicial", "RciInternoInicial", 
                             "ScoreInstallmentCat", "ScoreInstallment"]]
todos.drop_duplicates(subset="DPI", inplace=True)

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_131076\1350161219.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todos.drop_duplicates(subset="DPI", inplace=True)


In [7]:
todos["DPI"] = todos["DPI"].astype("int64")
todos.head(3)

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_131076\3240633409.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todos["DPI"] = todos["DPI"].astype("int64")


,DPI,SaldoExterno,SaldoInterno,RciGlobalInicial,RciInternoInicial,ScoreInstallmentCat,ScoreInstallment
0,1572859871309,0.00,13725.07,33.10,32.41,VeryLow,787.0
1,1573036832101,13842.59,169464.56,61.74,54.01,High,508.0
3,1573111121603,0.00,158096.33,46.37,46.37,Low,689.0


In [8]:
def rango_rci(interno: bool, rci: float) -> str:
    if interno: 
        if 0 <= rci and rci <= 10:
            return "0 <= x <= 10"
        if 10 < rci and rci <= 20:
            return "10 < x <= 20"
        if 20 < rci and rci <= 30:
            return "20 < x <= 30"
        if 30 < rci and rci <= 40:
            return "30 < x <= 40"
        if 40 < rci and rci <= 50:
            return "40 < x <= 50"
        if 50 < rci and rci <= 60:
            return "50 < x <= 60"
        elif rci > 60: 
            return "x > 60"
    elif not interno:
        if 0 <= rci and rci <= 10:
            return "0 <= x <= 10"
        if 10 < rci and rci <= 20:
            return "10 < x <= 20"
        if 20 < rci and rci <= 30:
            return "20 < x <= 30"
        if 30 < rci and rci <= 40:
            return "30 < x <= 40"
        if 40 < rci and rci <= 50:
            return "40 < x <= 50"
        if 50 < rci and rci <= 60:
            return "50 < x <= 60"
        if 60 < rci and rci <= 70:
            return "60 < x <= 70"
        if 70 < rci and rci <= 80:
            return "70 < x <= 80"
        elif rci > 80: 
            return "x > 80"
        
        

In [9]:
todos["RangoRciInterno"] = todos.apply(lambda x: rango_rci(True, x["RciInternoInicial"]), axis=1)
todos["RangoRciGlobal"] = todos.apply(lambda x: rango_rci(False, x["RciGlobalInicial"]), axis=1)
todos.head(3)

C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_131076\1707697900.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todos["RangoRciInterno"] = todos.apply(lambda x: rango_rci(True, x["RciInternoInicial"]), axis=1)
C:\Users\mariajose_chinchilla\AppData\Local\Temp\ipykernel_131076\1707697900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  todos["RangoRciGlobal"] = todos.apply(lambda x: rango_rci(False, x["RciGlobalInicial"]), axis=1)


,DPI,SaldoExterno,SaldoInterno,RciGlobalInicial,RciInternoInicial,ScoreInstallmentCat,ScoreInstallment,RangoRciInterno,RangoRciGlobal
0,1572859871309,0.00,13725.07,33.10,32.41,VeryLow,787.0,30 < x <= 40,30 < x <= 40
1,1573036832101,13842.59,169464.56,61.74,54.01,High,508.0,50 < x <= 60,60 < x <= 70
3,1573111121603,0.00,158096.33,46.37,46.37,Low,689.0,40 < x <= 50,40 < x <= 50


In [10]:
pd.crosstab(todos["RangoRciInterno"], todos["RangoRciGlobal"],).to_excel("../db/LaTeX/clientes.xlsx")

In [11]:
# Sacar saldo
pd.crosstab(todos["RangoRciInterno"], todos["RangoRciGlobal"],
            todos["SaldoInterno"], 
            aggfunc="sum").to_excel("../db/LaTeX/saldo_interno.xlsx")

In [12]:
# Sacar saldo
pd.crosstab(todos["RangoRciInterno"], todos["RangoRciGlobal"],
            todos["SaldoExterno"], 
            aggfunc="sum").to_excel("../db/LaTeX/saldo_externo.xlsx")